In [1]:
import pandas as pd

In [2]:
def pricing_gas_contract(injection_dates, withdrawal_dates, purchase_prices, sell_prices, 
                       injection_rates, withdrawal_rates, max_storage_volume, storage_costs):
    
    # We sort dates in order to easily calculate injection and withdrawal prices
    injection_dates = pd.to_datetime(injection_dates, format='%m/%d/%y')
    withdrawal_dates = pd.to_datetime(withdrawal_dates, format='%m/%d/%y')

    purchase_cost = sum([rate * price for rate, price in zip(injection_rates, purchase_prices)])
    
    sell_revenue = sum([rate * price for rate, price in zip(withdrawal_rates, sell_prices)])

    # Storage costs w.r.t withdrawal dates and injection dates 
    storage_cost = 0
    cumulative_volume = 0  
    
    for i, injection_date in enumerate(injection_dates):
        gas_injected = injection_rates[i]
        corresponding_withdrawal_date = withdrawal_dates[i]

       
        storage_days = (corresponding_withdrawal_date - injection_date).days 
        # We verify that the gas can be injected without risking a volume exceed
        
        if cumulative_volume + gas_injected > max_storage_volume:
            raise ValueError("Cumulative gas injected exceeds maximum storage volume")

        
        storage_cost += gas_injected * storage_days * storage_costs # We calculate the storage cost 

       
        cumulative_volume += gas_injected  # We update the gas volume on storage

        
        cumulative_volume -= withdrawal_rates[i] # We update the gas volume on storage

   
    contract_value = sell_revenue - purchase_cost - storage_cost  # We calculate the contract value ( added value for buyer)

    return contract_value



From a fundamental analysis and ecocomical point of view, a good strategy to follow on winter season is to inject a great amount of gas in prewinter period and then slow down the injection rates in favor of a big withdrawal. The idea is to take advantage of gas price increase in order to make profit. We will use this reasoning to define injection and withdrawal rates with respect to dates.

In [4]:
# Let's test our code
# We import historical price data and consider the last months of 2020 and first months of 2021 (mostly pre winter and winter season)
# The injection and withdrawal rates we take into consideration are not realistic. In fact a 10^(6) factor should be applied on a large scale

data_gas_prices=pd.read_csv('Nat_Gas.csv')
injection_dates= data_gas_prices['Dates'].iloc[[1,3,5,7]].to_list()
withdrawal_dates= data_gas_prices['Dates'].iloc[[2,4,6,8]].to_list()
purchase_prices = data_gas_prices['Prices'].iloc[[1,3,5,7]].to_list()
sell_prices = data_gas_prices['Prices'].iloc[[2,4,6,8]].to_list()
injection_rates=[200, 25, 25, 50 ] # Here we supposed that injection rates start big because winter is coming and decrease gradually because the demand
# starts to get the upperhand on the offer

withdrawal_rates=[25, 125, 125, 25] # Here we supposed that withdrawals rates start small but increase gradually because the demand start to get the upperhand

max_storage_volume=300

storage_costs= 0.001 # storage cost per unit per day ( should be much lower than the market price of natural gas unit) 


In [5]:
# Let's calculate the contract value for the scenario  we defined
contract_value=pricing_gas_contract(injection_dates, withdrawal_dates, purchase_prices, sell_prices, injection_rates, withdrawal_rates, max_storage_volume, storage_costs)

In [6]:
print(contract_value)

81.35
